In [6]:
import polars as pl

from nwec.constants import PROJECT_ROOT

In [8]:
df = pl.read_excel(f"{PROJECT_ROOT}/data/raw_data.xlsx", sheet_id=2, has_header=False)

# Number of Disconnections

In [ ]:
disconnects = df.select(df.columns[:15]).slice(1)
# Drop null columns and cast all columns to strings
null_columns = [col for col in disconnects.columns if disconnects[col][0] is None]
disconnects = disconnects.drop(null_columns)
disconnects = disconnects.with_columns([pl.col(col).cast(pl.Utf8) for col in disconnects.columns])
# Rename columns with proper headers
disconnects = disconnects.rename(disconnects.head(1).to_dicts()[0])
disconnects = disconnects.filter(pl.col("Zip Code") == "Residential Total")
disconnects = disconnects.with_columns([pl.col(col).cast(pl.Int32, strict=False) for col in disconnects.columns[2:]])
disconnects = disconnects.fill_null(0)

In [ ]:
disconnects

Zip Code,Customer Class,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Residential Total""",null,705,1013,1244,1164,1320,1081,0,0,0,0,0,0


# Disconnection Notices

In [ ]:
disconnect_notices = df.select([df.columns[0]] + df.columns[41:53]).slice(1)
# Drop null columns and cast all columns to strings
null_columns = [col for col in disconnect_notices.columns if disconnect_notices[col][0] is None]
disconnect_notices = disconnect_notices.drop(null_columns)
disconnect_notices = disconnect_notices.with_columns([pl.col(col).cast(pl.Utf8) for col in disconnect_notices.columns])
# Rename columns with proper headers
disconnect_notices = disconnect_notices.rename(disconnect_notices.head(1).to_dicts()[0])
disconnect_notices = disconnect_notices.filter(pl.col("Zip Code") == "Residential Total")
disconnect_notices = disconnect_notices.with_columns(
    [pl.col(col).cast(pl.Int32, strict=False) for col in disconnect_notices.columns[1:]]
)
disconnect_notices = disconnect_notices.fill_null(0)

In [ ]:
disconnect_notices

Zip Code,2024-01-01 00:00:00,2024-02-01 00:00:00,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Residential Total""",6327,5851,3775,9144,8053,5644,0,0,0,0,0,0
